Before you turn in this assignment, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then run the test cells for each of the questions you have answered.  Note that a grade of 3 for the related LOs requires all tests in the "Basic Functionality" section to be passed.  The test cells pass if they execute with no errors (i.e. all the assertions are passed).

Make sure you fill in any place that says `YOUR CODE HERE`.  Be sure to remove the `raise NotImplementedError()` statements as you implement your code - these are simply there as a reminder if you forget to add code where it's needed.

---

<h1>CS152 Assignment 2: The DPLL Algorithm</h1>

<h1>Question 1</h1>

Define your <code>Literal</code> class below.  Ensure that you define a <code>name</code> and <code>sign</code> attribute as required in the assignment description.  In addition, include any other attributes and member functions as needed. You will need to overload <code>__neg__()</code> to correctly handle negated literals.  You may need to overload <code>__eq__()</code> and <code>__hash__()</code> also.

In [1]:
class Literal:

    def __init__(self, name: str) -> None:
        self.name = name
        self.sign = True  # Initialize the literal sign as positive

    def pure(self):
        """Gets the pure version of the Literal without any negative"""
        if self.sign:
            return self
        else:
            return -self

    def __neg__(self):
        """Flip the sign of the Literal"""
        l = Literal(self.name)
        l.sign = not self.sign
        return l

    def __repr__(self) -> str:
        sign = "" if self.sign else "-"
        return f"{sign}{self.name}"

    def __eq__(self, o: object) -> bool:
        """Two literals are equal when their names are equal"""
        return self.name == o.name

    def __hash__(self) -> int:
        return self.name.__hash__()


<h1>Question 2</h1>
Implement DPLL by filling in the stubs below.  Note that the various heuristics are not required to be implemented for basic functionality, but a template has been provided for you to do so if you attempt the extension questions

In [10]:
from typing import List, Set, Tuple, Dict


def simplify_clauses(
    clauses: List[Set[Literal]], 
    model: Dict[Literal, bool]
    ) -> List[Set[Literal]]:
    """Simplifies the KB based on the model provided"""
    
    def simplify_clause(
        clause: Set[Literal], 
        model: Dict[Literal, bool]
        ) -> Set[Literal]:
            """Simplifies a clause based on the model provided"""
            new_clause: Set[Literal] = set()
            for literal in clause:
                current_evaluation = model.get(literal, None)
                if current_evaluation and literal.sign or current_evaluation == False and literal.sign == False:
                    return None  # Returns nothing
                elif current_evaluation == None:
                    new_clause.add(literal)
            # Return the original clause if all the values are False
            if len(new_clause) == 0: return clause  
            return new_clause

    new_clauses: List[Set[Literal]] = []
    for clause in clauses:
        new_clause = simplify_clause(clause, model)
        if new_clause is not None:
            new_clauses.append(new_clause)
    return new_clauses


def get_symbols(clauses: List[Set[Literal]]) -> Set[str]:
    """Returns the symbols in the clauses"""
    symbols: Set[str] = set()
    for clause in clauses:
        for literal in clause:
            symbols.add(literal)
    return symbols


def evaluate_clauses(
    clauses: List[Set[Literal]], 
    model: Dict[Literal, bool]
    ) -> bool:
    """Checks if all the clauses evaluate to true, returning true, or if any
    evaluates to false, returning false, or if the status cannot be told, None"""
    
    current_clauses: List[Set[Literal]] = simplify_clauses(clauses, model)

    # When every clause in clauses is True in the model
    if len(current_clauses) == 0: return True

    for clause in current_clauses:
        # When all the literals in the clause are False, the clause is False
        all_false: bool  = True
        for literal in clause:
            if model.get(literal, None) is None:
                all_false = False
                break
        if all_false:
            return False
    return None

def most_common_symbol(
    symbols: Set[Literal], 
    clauses: List[Set[Literal]],
    model:Dict[Literal, bool]
    ) -> Tuple[Literal, Set[Literal]]:

    current_clauses: List[Set[Literal]] = simplify_clauses(clauses, model)

    counts = {}
    highest: Literal = None
    for clause in current_clauses:
        for literal in clause:
            counts[literal] = counts.get(literal, 0)
            if highest is None:
                highest = literal
            else:
                if counts[literal] > counts[highest]:
                    highest = literal
                elif counts[literal] == counts[highest]:
                    highest = literal if literal.name < highest.name else highest
                else:
                    pass
    new_symbols = symbols.copy()
    new_symbols.remove(highest)
    return highest, new_symbols


def DPLL_Satisfiable(
    KB: List[Set[Literal]], 
    heuristic_level=0
    ) -> Tuple[bool, Dict[Literal, bool], List[Literal]]:
    ''' satisfiable, model = DPLLSatisfiable(KB)
        Takes in a KB and returns whether the KB is satisfiable, and the model that makes it so
        
        KB: A knowledge base of clauses (CNF) consisting of a list of sets of literals.  A KB might look like
            [{A,B},{-A,C,D}]
        heuristic_level: An integer that will be passed in to specify which heuristics to implement 
            (only for the extension activities).
        satisfiable: Returns True if the KB is satisfiable, or False otherwise
        Model: A dictionary that assigns a truth value to each literal for the model that satisfies KB.
            For example, a model might look like {A: True, B: False}
    '''
    def DPLL(
        clauses: List[Set[Literal]], 
        symbols: Set[Literal], 
        model:Dict[Literal, bool],
        heuristic_level: int = 0,
        symbol_list: List[Literal] = []
        ) -> Tuple[bool, Dict[Literal, bool], List[Literal]]:
        """
        Takes in a KB and recursively tries to figure out whether the KB is satisfiable, 
        and the model that makes it so.
            
        :param clauses: A knowledge base of clauses (CNF) consisting of a list of sets of literals.  A KB might look like
                [{A,B},{-A,C,D}]
        :param symbols: The symbols present in the KB
        :param model: The model to use during the search
        :param heuristic_level: An integer that will be passed in to specify which heuristics to implement 
                (only for the extension activities).
                0: No heuristic is used
                1: The degree heuristic
                2: Pure symbol and unit clause heuristics
                3: Most occuring pure symbol
        :param symbol_list: The currently chosen symbols. It should be initialized as empty
        :returns satisfiable: Returns True if the KB is satisfiable, or False otherwise
        :returns Model: A dictionary that assigns a truth value to each literal for the model that satisfies KB.
                For example, a model might look like {A: True, B: False}
        :returns symbol_list: symbols chosen for assignment
        """

        evaluation: bool = evaluate_clauses(clauses, model)

        if evaluation is not None:
            return evaluation, model, symbol_list

        if heuristic_level == 1:
            # Degree heuristic - Most common symbol first
            symbols_lst = list(symbols)
            P, rest = most_common_symbol(symbols, clauses, model) 

            symbol_list.append(P.pure())
            when_true = DPLL(clauses, rest, {**model, **{P.pure(): True}}, heuristic_level)
            if when_true[0]:
                return when_true

            symbol_list.append(P.pure())
            when_false = DPLL(clauses, rest, {**model, **{P.pure(): False}}, heuristic_level)
            return when_false
        elif heuristic_level == 2:
            P, value = find_pure_symbol(symbols, clauses, model)
            if P: 
                new_symbols = symbols.copy()
                new_symbols.remove(P)
                symbol_list.append(P.pure())
                return DPLL(clauses, new_symbols, {**model, **{P.pure(): value}}, heuristic_level)
            P, value = find_unit_clause(clauses, model)
            if P: 
                new_symbols = symbols.copy()
                new_symbols.remove(P)
                symbol_list.append(P.pure())
                return DPLL(clauses, new_symbols, {**model, **{P.pure(): value}}, heuristic_level)
        elif heuristic_level == 3:
            pass
        else:  # heuristic_level assumed to be 0
            symbols_lst = list(symbols)
            P, rest = symbols_lst[0], set(symbols_lst[1:])

            symbol_list.append(P.pure())
            when_true = DPLL(clauses, rest, {**model, **{P.pure(): True}}, heuristic_level)
            if when_true[0]:
                return when_true

            symbol_list.append(P.pure())
            when_false = DPLL(clauses, rest, {**model, **{P.pure(): False}}, heuristic_level)
            return when_false
 
    clauses = KB
    symbols: List[str] = get_symbols(clauses)
    return DPLL(clauses, symbols, {}, heuristic_level)


def find_pure_symbol(
    symbols: Set[Literal], 
    clauses: List[Set[Literal]],
    model:Dict[Literal, bool]
    ) -> Tuple[Literal, bool]:
    """
    Finds a pure symbol in the current state of the KB (clauses).
    """
    current_clauses: List[Set[Literal]] = simplify_clauses(clauses, model)

    pure_symbols: Set[Literal] = set()
    # Stores the sign a symbol should have to be pure 
    symbol_signs: Dict[Literal, bool] = {}
    symbols_lst = list(symbols) 
    for i in range(len(symbols)):
        symbol = symbols_lst[i]
        for clause in current_clauses:
            if symbol in clause:
                clause_lst = list(clause)
                literal = clause_lst[clause_lst.index(symbol)]
                if symbol in symbol_signs:
                    if literal.sign != symbol_signs[symbol]:
                        pure_symbols.remove(symbol)
                        i += 1
                        break
                else:
                    symbol_signs[symbol] = literal.sign
                    pure_symbols.add(symbol)

    pure_symbol =  list(pure_symbols)[0] if len(pure_symbols) > 0 else None

    if pure_symbol is None:
        return None, None

    return pure_symbol, symbol_signs[pure_symbol]
                

def find_unit_clause(
    clauses: List[Set[Literal]],
    model:Dict[Literal, bool]
) -> tuple:

    current_clauses: List[Set[Literal]] = simplify_clauses(clauses, model)

    for clause in current_clauses:
        if len(clause) == 1:
            literal = clause.pop()
            return literal, literal.sign
    
    return None, None

<h1>Question 3</h1>

Implement your KB from Russell & Norvig in CNF by filling in the sets inside the list <code>KB</code> below.  Ensure that your KB is in a list of set format as stated in the assignment instructions.

In [11]:
# Define some literals
A = Literal('A')
B = Literal('B')
C = Literal('C')
D = Literal('D')
E = Literal('E')
F = Literal('F')

KB = [
    {-A, B, E}, {A, -B}, {A, -E},   # S1: A <=> (B v E)
    {-E, D},                        # S2: E => D
    {-C, -F, -B},                   # S3: C ^ F => ~B
    {-E, B},                        # S4: E => B
    {-B, F},                        # S5: B => F 
    {-B, C}                         # S6: B => C
]

<h1>Extensions</h1>

1. Implement the degree heuristic for choosing symbols.  If <code>heuristic_level=1</code>, then the degree heuristic should be used to select which symbol to assign.
2. Implement the pure symbol and unit clause heuristics.  If <code>heuristic_level=2</code>, then these heuristics should be used to find select first a pure symbol, and if one is not found, then a unit clause.  If neither pure symbols nor unit clauses are found, then the degree heuristic should be used to select a symbol.  If there are multiple pure symbols or unit clauses found, then use alphabetical order to select amongst them.
3. Modify the pure symbol heuristic to choose the pure symbol that occurs in the most number of clauses.  This should be activated if <code>heuristic_level=3</code>.

<h1>Basic Functionality Tests</h1>

All of the tests in this section must be passed for the code to be considered fully functional.  Other unseen test cases will be used after submission to further verify functionality

In [12]:
# This section will test the correct definition of the literal class


# Test the name attribute works correctly
assert(A.name == 'A')

# Test that negation works correctly
assert(not (-C).sign)

In [13]:
# This section will test that the KB has been correctly converted to CNF by performing 
# tests over all possible models
import itertools
symbols = {A,B,C,D,E,F}
symbol_list = [A,B,C,D,E,F]

def evaluate_russell_norvig_KB(model):
    # Manually evaluate the KB using the model
    s = list()
    s.append(model[A] == (model[B] or model[E]))
    s.append(model[E] <= model[D])
    s.append((model[C] and model[F]) <= (not model[B]))
    s.append(model[E] <= model[B])
    s.append(model[B] <= model[F])
    s.append(model[B] <= model[C])
    return all(s)

def evaluate_KB(KB, model):
    model_true = True
    model_name_dict = {l.name: model[l] for l in model}
    for clause in KB:
        evaluation = {l.sign == model_name_dict[l.name] for l in clause if l.name in model_name_dict}
        model_true = model_true and any(evaluation)
    return model_true

all_models = list(itertools.product([False, True], repeat=6))
for i in range(0, len(all_models)):
    # Test all truth values
    test_model = {symbol_list[j]: all_models[i][j] for j in range(0,6)}
    
    # Test the model
    assert(evaluate_russell_norvig_KB(test_model) == evaluate_KB(KB, test_model))

In [14]:
# This section will test the basic working of DPLL
# Satisfiable model
test_KB = [{A, C},{-A, C}, {B, -C}]
t, model, symbol_trace = DPLL_Satisfiable(test_KB)
assert(evaluate_KB(test_KB, model))
assert(t)

# Unsatisfiable model
test_KB = [{A, C},{-A, C}, {-C}]
t, model, symbol_trace = DPLL_Satisfiable(test_KB)
assert(not t)

In [15]:
# This will test DPLL on the KB from Russell & Norvig
t, model, symbol_trace = DPLL_Satisfiable(KB,0)
print(model)

# This model is satisfiable.  Test that it is so
assert(t)

# Check that the model found actually works
assert(evaluate_KB(KB, model)) 

{C: True, A: False, B: False, F: True, E: False}


<h1>Extension Tests</h1>

This section will test that the degree heuristic, pure symbol and unit clause heuristics are correctly implemented.  Note that in your code, the <code>heuristic_level</code> needs to correctly activate the heuristic being tested, and the <code>symbol_list</code> needs to be correctly generated 

In [16]:
# Degree Heuristic Test for KB from Russell & Norvig
t, model, symbol_trace = DPLL_Satisfiable(KB,1)
print(model)

# This model is satisfiable.  Test that it is so
assert(t)

# Check that the model found actually works
assert(evaluate_KB(KB, model))  

# Test the symbol trace to ensure that the correct order is chosen
assert([l.name for l in symbol_trace] in [['B','F', 'E', 'C', 'C', 'E', 'A'], ['B', 'A', 'C', 'F', 'E', 'A']])

symbol_trace

{A: False, B: False, D: True, E: False}


AssertionError: 

In [17]:
print(symbol_trace)

[A, B, C, D, F, F, D, E, E, F, F, C, B, D, E, E, D, E, E, A, B, B, D, E, E]


In [ ]:
# Pure Symbol & Unit Clause Heuristic Test for KB from Russell & Norvig
t, model, symbol_trace = DPLL_Satisfiable(KB,2)
print(model)


# This model is satisfiable.  Test that it is so
assert(t)

# Check that the model found actually works
assert(evaluate_KB(KB, model))

# Test the symbol trace to ensure that the correct order is chosen
assert([l.name for l in symbol_trace] == ['D', 'B', 'E', 'A', 'C', 'F', 'E', 'A'])

In [ ]:
# Pure Symbol & Unit Clause Heuristic Test, choosing the most-frequently used pure symbol, for KB from Russell & Norvig
t, model, symbol_trace = DPLL_Satisfiable(KB,3)
print(model)
print(symbol_trace)

# This model is satisfiable.  Test that it is so
assert(t)

# Check that the model found actually works
assert(evaluate_KB(KB, model))

# Test the symbol trace to ensure that the correct order is chosen
assert([l.name for l in symbol_trace] == ['D', 'B', 'A', 'C', 'F', 'E', 'A'])